In [19]:
import pandas as pd
import re

In [20]:
df=pd.read_csv('D:\Projects\DATA_ANALYST_PROJECTS\web_scraping_job_market_analysis\data/naukri_data_analyst_jobs.csv')

In [21]:
df.head()

,Job Title,Company Name,Experience Required,Salary,Location,Posted
0,Data Analyst,Infineon,0-2 Yrs,Not disclosed,Bengaluru,1 Day Ago
1,Offshore Data Analyst,"NTT DATA, Inc.",3-5 Yrs,Not disclosed,Bengaluru,Few Hours Ago
2,Data Analyst L3,Wipro,1-3 Yrs,Not disclosed,Bengaluru,1 Day Ago
3,"Python Developer, Data Analyst, - Year of Unde...",Codesphere Solutions,0-5 Yrs,8-18 Lacs PA,Hybrid - Bengaluru,2 Days Ago
4,Data Analyst - Python/Hadoop,Sadup Soft,3-6 Yrs,Not disclosed,Bengaluru,1 Day Ago


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Job Title            1000 non-null   object
 1   Company Name         1000 non-null   object
 2   Experience Required  1000 non-null   object
 3   Salary               1000 non-null   object
 4   Location             1000 non-null   object
 5   Posted               1000 non-null   object
dtypes: object(6)
memory usage: 47.0+ KB


In [23]:
df.drop_duplicates(inplace=True)

In [24]:
df.dropna(inplace=True)

In [25]:
df['Experience Required'] = df['Experience Required'].replace(
    to_replace=r'\d{2} [A-Za-z]{3} - \d{2} [A-Za-z]{3}', #\d{2} = two digits (like 09) ,[A-Za-z]{3} = three letters (like Apr)
    value=0, 
    regex=True
)

In [26]:
df['Experience Required']=df['Experience Required'].replace({'3 months duration':0.25,'6 months duration':0.5,'No fixed duration':0})

In [27]:
df['Experience Required']=df['Experience Required'].replace(to_replace='Yrs',value='',regex=True)

In [34]:
df['Salary'].unique()

array(['Not disclosed', '800000-1800000', '325000-800000',
       '175000-300000', '175000-400000', '50000-200000',
       '1000000-1600000', '400000-700000', '1400000-2400000',
       '550000-1100000', '3500000-4000000', '1300000-1500000',
       '300000-450000', '225000-300000', '275000-550000', '450000-650000',
       '500000-700000', '300000-400000', '1200000-1600000',
       '1500000-1800000', '1200000-1500000', '1000000-1500000',
       '1500000-2000000', '175000-250000', '800000-1100000',
       '500000-800000', '300000-425000', '250000-375000',
       '1300000-1800000', '400000-800000', '400000-425000',
       '1100000-1900000', '900000-1700000', 'Unpaid', '950000-1950000',
       '800000-1500000', '600000-750000', '500000-1000000',
       '700000-1000000', '2500000-3500000', '400000-900000',
       '325000-650000', '200000-350000', '800000-1000000',
       '1000000-1200000', '1300000-1700000', '100000-400000',
       '475000-800000', '200000-500000', '225000-375000',
       '1

In [29]:
df['Salary']=df['Salary'].replace('(Including Variable: 10.0%)','',regex=True)
df['Salary']=df['Salary'].replace('(Including Variable: 30.0%)','',regex=True)

In [32]:
def convert_salary_range(salary):
    if isinstance(salary,str):      # is use to check we work in string format
        salary=salary.replace('PA','')
        salary=salary.replace('()','').strip()

    if '-' in salary:
        parts=salary.split('-')
        low=parts[0].strip()
        high=parts[1].strip()

        if ',' in low:
            low=low.replace(',','').strip()
        else:
            low=float(low)*100000

        if ',' in high:
            high=high.replace(',','').strip()
        else:
            high=float(high.replace('Lacs',''))*100000

        return str(int(low)) + '-' + str(int(high))
    
    elif '/month' in salary.lower():
        amounts=salary.replace('/month','').replace(',','').strip()
        return str(int(amounts)*12)
    
    else:
        return salary

In [33]:
df['Salary']=df['Salary'].apply(convert_salary_range)

In [90]:
df['Posted'].unique()

array(['1 Day Ago', 'Few Hours Ago', '2 Days Ago', '5 Days Ago',
       '7 Days Ago', '14 Days Ago', '8 Days Ago', '6 Days Ago',
       '15 Days Ago', '13 Days Ago', '12 Days Ago', '22 Days Ago',
       '29 Days Ago', '11 Days Ago', '30+ Days Ago', '4 Days Ago',
       '9 Days Ago', '16 Days Ago', '18 Days Ago', '19 Days Ago',
       '30 Days Ago', '21 Days Ago', 'Starts in 1-3 months',
       '27 Days Ago', '28 Days Ago', '17 Days Ago', '20 Days Ago',
       '24 Days Ago', '23 Days Ago', '25 Days Ago', '10 Days Ago',
       '26 Days Ago', "Starts : 17th Apr' 25", 'Just Now'], dtype=object)

In [91]:
df['Posted']=df['Posted'].replace({'Day Ago':'','Days Ago':'','Few Hours Ago':'today','Just Now':'today','Starts in 1-3 months':'later',"Starts : 17th Apr' 25":'later'},regex=True)

In [ ]:
df['Job Title']=df['Job Title'].apply(lambda x:x.split('-')[0].strip())
df['Job Title']=df['Job Title'].replace('Walk','MIS/Operations/Data Analyst')


In [93]:
df.head()

,Job Title,Company Name,Experience Required,Salary,Location,Posted
0,Data Analyst,Infineon,0-2,Not disclosed,Bengaluru,1
1,Offshore Data Analyst,"NTT DATA, Inc.",3-5,Not disclosed,Bengaluru,today
2,Data Analyst L3,Wipro,1-3,Not disclosed,Bengaluru,1
3,"Python Developer, Data Analyst,",Codesphere Solutions,0-5,800000-1800000,Hybrid - Bengaluru,2
4,Data Analyst,Sadup Soft,3-6,Not disclosed,Bengaluru,1


In [97]:
df['Location'].unique()

array(['Bengaluru', 'Hybrid - Bengaluru', 'Pune', 'Hisar', 'Remote',
       'Kolkata, Mumbai, New Delhi, Hyderabad, Pune, Chennai, Bengaluru',
       'Ahmedabad', 'Surat', 'Puducherry', 'Hyderabad', 'Gurugram',
       'Mumbai', 'Noida', 'Hybrid - Noida, Hyderabad, Gurugram',
       'Kolkata', 'Hybrid - Mumbai (All Areas)(Powai)', 'Goa',
       'New Delhi', 'Ankleshwar', 'Mumbai, Delhi / NCR, Bengaluru',
       'Hyderabad, Bengaluru', 'Hybrid - Hyderabad, Chennai, Bengaluru',
       'Hyderabad(Gachibowli)', 'Chandigarh',
       'Hyderabad, Chennai, Bengaluru', 'Chennai, Gurugram, Bengaluru',
       'Bengaluru(Bannerghatta Road)', 'Pune, Bengaluru',
       'Bengaluru(HSR Layout)', 'Hybrid - Coimbatore', 'India',
       'New Delhi(Kirti Nagar)', 'Hybrid - Pune(Kalyani Nagar)',
       'Chennai', 'Hybrid - Hyderabad', 'Mumbai, Hyderabad',
       'Hyderabad, Gurugram', 'Hybrid - Hyderabad, Bengaluru',
       'Mumbai, Bengaluru', 'Hybrid - Kochi, Visakhapatnam, Coimbatore',
       'Hybrid - K